In [1]:
#importando las librerías de lectura
import numpy as np
import pandas as pd

/home/carlos/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
#importando las librerías especializadas
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.svm import SVC

In [119]:
#Creando clases para normalizar una región de la matriz
class Normalizer:
    matrix = []
    m = 0
    n = 0
    
    def __init__(self, matrix):
        self.matrix = matrix
        self.m,self.n = matrix.shape
    
    def getKeyValuesFromColumn(self,colIdx):
        values = np.unique(self.matrix[:,colIdx])
        keyValues = {}
        for val in values:
            keyValues[val] = len(keyValues)
        return keyValues
        
    def changeColumnToNumericInterval(self,colIdx):
        keyValues = self.getKeyValuesFromColumn(colIdx)
            
        for i in range(self.m):
            self.matrix[i][colIdx] = keyValues[self.matrix[i][colIdx]]
            
    
    def toStandard(self, colIdx):
        col = self.matrix[:,colIdx]
        mean = np.mean(col)
        std = np.std(col)
        
        self.matrix[:,colIdx]-=mean
        self.matrix[:,colIdx]/=std
        
    def getMatrix(self):
        return np.asarray(self.matrix, dtype=float)
    

In [91]:
#Leyendo X , Y
X = pd.read_csv('x.csv', sep=';')
Y = pd.read_csv('y.csv', sep=';')
print(X.shape)
print(Y.shape)

(591, 20)
(591, 1)


In [121]:
#Normalizando los datos
xNorm = Normalizer(X.values)
yNorm = Normalizer(Y.values)


#Normalizando X
xNorm.changeColumnToNumericInterval(0) #salon
xNorm.changeColumnToNumericInterval(1) #colegio
xNorm.changeColumnToNumericInterval(2) #sexo
xNorm.changeColumnToNumericInterval(3) #Edad
xNorm.changeColumnToNumericInterval(19) #Interpretación


xNorm.toStandard(7) #latitud
xNorm.toStandard(8) #longitud
xNorm.toStandard(18) #Puntaje Logrado

#Normalizando Y
yNorm.changeColumnToNumericInterval(0)

In [129]:
#Entrenando el modelo SVM
from sklearn.svm import SVC
clf = SVC(gamma='auto')
newX, newY = xNorm.getMatrix(), yNorm.getMatrix()
m,n = newX.shape

#Entrenando
newY = newY.ravel();
clf.fit(newX, newY) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [130]:
#Probando consigo mismo
predictedY = clf.predict(newX)
precision = 0
for i in range(m):
    if predictedY[i] == newY[i]:
        precision+=1

print("Precision consigo mismo: ", (precision*100.0/m))

Precision consigo mismo:  54.653130287648054
